In [ ]:
import pmdarima as pm
#import auto_arima # requires numpy version 1.26.4 i.e. before 2. 
import time
import pandas as pd

In [1]:
datetime = "2025-03-19_10-47-56"
filename_DD = f'../data/nextbike/hourly_demand_supply_Dresden {datetime}.csv'
filename_FB = f'../data/nextbike/hourly_demand_supply_Freiburg {datetime}.csv'
df_DD = pd.read_csv(filename_DD, index_col=None, parse_dates=['datetime_hour'])
df_FB = pd.read_csv(filename_FB, index_col=None, parse_dates=['datetime_hour'])

NameError: name 'pd' is not defined

In [ ]:
mycell='871f1b559ffffff'

In [2]:
df_DD.rent_count.sum()

NameError: name 'df_DD' is not defined

In [ ]:
df_DD.hex_id.unique()

In [ ]:
df_FB.hex_id.unique()

In [ ]:
df_DD.loc[df_DD.hex_id==mycell, ['datetime_hour', 'rent_count', 'return_count']].plot(x='datetime_hour', y=['rent_count', 'return_count'])

In [ ]:
test_range_1_DD = pd.date_range(start='2024-03-21', end='2024-03-31')
test_range_1_DD = [date.date() for date in test_range_1_DD]

In [ ]:
test_range_2_DD = pd.date_range(start='2024-10-21', end='2024-10-31')
test_range_2_DD = [date.date() for date in test_range_2_DD]


In [ ]:
df_DD_1 = df_DD.loc[df_DD.datetime_hour.dt.date <= test_range_1_DD[-1]]

In [ ]:
# df_DD_1.plot(x='datetime_hour', y=['rent_count', 'return_count'])

In [ ]:
df_DD_2 = df_DD.loc[df_DD.datetime_hour.dt.date > test_range_1_DD[-1]]

In [ ]:
# df_DD_2.plot(x='datetime_hour', y=['rent_count', 'return_count'])

In [ ]:
# df_DD_1_for_acf = df_DD_1.groupby('datetime_hour').agg({'rent_count': 'sum', 'return_count': 'sum'})

In [ ]:
# df_DD_2_for_acf = df_DD_2.groupby('datetime_hour').agg({'rent_count': 'sum', 'return_count': 'sum'})

In [ ]:
# df_DD_for_acf = df_DD.groupby('datetime_hour').agg({'rent_count': 'sum', 'return_count': 'sum'})

In [ ]:
# pm.utils.acf(df_DD_for_acf.rent_count, nlags=25)

In [ ]:
# df_FB_for_acf = df_FB.groupby('datetime_hour').agg({'rent_count': 'sum', 'return_count': 'sum'})

In [ ]:
# pm.tsdisplay(df_DD_2_for_acf.rent_count, lag_max=24*3, title="Dresden", show=True)

In [ ]:
# pm.tsdisplay(df_DD_for_acf.return_count, lag_max=24*3, title="Dresden bike sharing supply", show=True)
# pm.tsdisplay(df_FB_for_acf.return_count, lag_max=24*3, title="Freiburg bike sharing supply", show=True)

In [ ]:
# pm.tsdisplay(df_DD_for_acf.rent_count, lag_max=24*3, title="Dresden bike sharing demand", show=True)
# pm.tsdisplay(df_FB_for_acf.rent_count, lag_max=24*3, title="Freiburg bike sharing demand", show=True)

In [ ]:
# pm.tsdisplay(df_DD_1_for_acf.rent_count, lag_max=24*3, title="Dresden", show=True)

In [ ]:
# df_DD.loc[df_DD.datetime_hour.dt.date.isin(test_range_DD) & (df_DD.hex_id==mycell)].sort_values('datetime_hour')

In [ ]:
train_validation_DD_1 = df_DD_1.loc[~df_DD_1.datetime_hour.dt.date.isin(test_range_1_DD)]

In [ ]:
test_DD_1 = df_DD.loc[df_DD.datetime_hour.dt.date.isin(test_range_1_DD)].sort_values('datetime_hour')

In [ ]:
train_validation_DD_2 = df_DD_2.loc[~df_DD_2.datetime_hour.dt.date.isin(test_range_2_DD)].sort_values('datetime_hour')

In [ ]:
test_DD_2 = df_DD_2.loc[df_DD_2.datetime_hour.dt.date.isin(test_range_2_DD)].sort_values('datetime_hour')

In [ ]:
len(test_DD_2)

In [ ]:
len(train_validation_DD_2)

In [ ]:
train_1 = train_validation_DD_1[train_validation_DD_1.hex_id==mycell].set_index('datetime_hour')['rent_count']

In [ ]:
test_1 = test_DD_1[test_DD_1.hex_id==mycell].set_index('datetime_hour')['rent_count']

In [ ]:
train_1 = train_1.asfreq('h')

In [ ]:
# test_1.asfreq('h').isna().value_counts()

In [ ]:
test_1.asfreq('h').isna().value_counts()

In [ ]:
test_1 = test_1.asfreq('h')

In [ ]:
start_train_time = time.time()
model = pm.ARIMA(order=(0, 1, 1), seasonal_order=(0, 0, 2, 24))
model.fit(train_1)

print(f"Elapsed time: {(time.time() - start_train_time)/60} minutes")

In [ ]:
model.summary()

In [ ]:
predictions = model.predict(n_periods=264)

In [ ]:
predictions

In [ ]:
test_1

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
x = test_1.index
plt.plot(x, test_1, color='black')
plt.scatter(x, predictions, color='black')
plt.show()